In [1]:
from omegaconf import OmegaConf
import sentencepiece


In [2]:
conf = {'processed_path': "./processed_data", "split": "train", "context_length": 512, 'stride_big_sequences': 256, "count_past_boards": 2, "in_memory":False, 'target_types': [0, 1, 2, 3, 4]}
oconf = OmegaConf.create(conf)

In [3]:
sp = sentencepiece.SentencePieceProcessor("./artifacts/old/artifacts_old/artifacts/sp1000.model")

In [4]:
from data.AlphazeroCommentaryDataset import AlphazeroCommentaryDataset

In [5]:
ds = AlphazeroCommentaryDataset(oconf, sp)

In [6]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader

def collate_fn(data):
    board_data = torch.stack(list(map(lambda x: x[0], data)))
    sequences = pad_sequence(list(map(lambda x: x[1], data)), batch_first=True,
                             padding_value=sp.pad_id())
    types = torch.stack(list(map(lambda x: x[2], data)))
    X_sequence = sequences[:, :-1]
    y_sequence = sequences[:, 1:]
    next_pad_mask = (y_sequence == sp.pad_id())
    return board_data, X_sequence, y_sequence, next_pad_mask, types

dl = DataLoader(
            ds,
            batch_size=128,
            collate_fn=collate_fn,
            num_workers=2
        )

In [17]:
import wandb
run = wandb.init()
artifact = run.use_artifact('georgerapeanu/thesis/model_val_loss:v68', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: georgerapeanu. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [7]:
model = torch.load("./artifacts/model_val_loss:v68/model_save_val_loss_2.341321006943198", map_location=torch.device("cpu"))
model.eval()

Model(
  (board_preparation): Sequential(
    (0): Conv2d(125, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (emb): Embedding(1000, 256)
  (pe_board): PositionalEncoding2D(
    (Dropout): Dropout(p=0.1, inplace=False)
  )
  (pe_text): PositionalEncoding1D(
    (Dropout): Dropout(p=0.1, inplace=False)
  )
  (encoders): ModuleList(
    (0-5): 6 x TransformerEncoderBlock(
      (mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (layer_norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=512, out_features=256, bias=True)
      )
      (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
   

In [8]:
import random

choices = random.sample(range(len(ds)), 10)
to_predict = [ds[i] for i in choices]
to_predict_metadata = [ds.get_raw_data(i) for i in choices]

In [9]:
from model.predict import Predictor
predictor = Predictor(512, sp)

In [13]:
for ((X_board, y_tokens, _), (current_board, past_board, current_eval, past_eval)) in zip(to_predict,
                                                                                                  to_predict_metadata):
    predicted_text = predictor.predict(model, X_board, '', 1024)
    actual_text = predictor.tokens_to_string(y_tokens)
    print("actual:" + actual_text)
    print("predicted:" + predicted_text)

actual:Seems to have over-extended in his desire to open lines to the king. Ng6, aiming for Nf4 is more patient and better. You should get your pieces into position first, then attack.
predicted:I wonder if this was much better. Then the added protection is to complicate white's own, manoeuvre vs. a4 seems better. But versus, has a plan to go, and play either quickly or to add the good support to the kingside attack. It's worth noting willing to push that pawn, but this will be a good attacking move and there are dynamic. Finishing viewed me had came in on.... I was alright as its feeling like all I had the need is illustrately... something didn't and I am badly bad, so I typically decided slowly-coverside pawns, that’s guarantees that you should ever get through the center after allowing any material, so White's pawn is on the open files. So, also it is already threatens as a discovered possibility and that should manage to initiate it.
actual:I keep the pressure on the pawn
predicted

In [12]:
torch.tensor([0]).device

device(type='cpu')

In [16]:
type(model)

model.commentary_models.Model

In [17]:
isinstance(model, torch.nn.Module)

True